# Gendering the talks 

One of our areas of interest are to see if TED talks by men and women are different. If there are differences, we would like to see if we can detect the gender of a speaker based on the words in a TED talk transcript. To pursue these avenues, we begin by "gendering" talks when possible. 

In this notebook, we use the genders of the speakers to "gender" the TED talks themselves. A talk with one speaker inherits the gender of the speaker. For a talk with two speakers, if the genders of the speakers are the same, then we proceed as if the talk had one speaker. For talks with two speakers where the genders of the speakers is not the same, we place these talks to the side. 

In [1]:
import pandas as pd
import csv
import string

In [2]:
# Load the gendered speaker file:
speakers = pd.read_csv("speakers_with_gender.csv")

In [20]:
ted_only = pd.read_csv('../data/Release_v0/TEDonly_final.csv')
ted_plus = pd.read_csv('../data/Release_v0/TEDplus_final.csv')

In [21]:
ted_only.head()

,Unnamed: 0,Unnamed: 0.1,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4
0,0,0,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN
1,1,1,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN
2,2,2,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN
3,3,3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN
4,4,4,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN


In [22]:
# Set the talk ID as the index and drop the unnecessary first two columns: 
ted_only = ted_only.set_index('Talk_ID')
ted_only = ted_only.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

In [23]:
ted_only.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4
Talk_ID,,,,,,,,,,,,,
1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN
7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN
53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN
66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN
92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN


In [ ]:
talk1speak = df[df['Col2'].isnull()]



# https://stackoverflow.com/questions/43831539/how-to-select-rows-with-nan-in-particular-column